# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [ ]:
# write your code
import requests

page = requests.get('https://www.sfu.ca/computing/people/faculty.html')
with open('csfaculty.html',mode='w') as writer:
    writer.write(page.content)

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [245]:
# write your code
from bs4 import BeautifulSoup
import re
import pandas as pd

pd.set_option('display.max_rows',1000)

with open('csfaculty.html',mode='r') as html_doc:

    soup = BeautifulSoup(html_doc,'html.parser')
    faculty_list = soup.find('div',class_='faculty-list')
    faculty_data = faculty_list.find_all('div',class_='textimage section')
    names_ = []
    ranks_ = []
    area_ = []
    profile_ = []
    homepage_ = []
    for row in faculty_data:
        name = row.h4.text
        if name.find("Fred") > 0:
            name = name.replace('\n','').strip()
            name = re.sub(r'[A-z]+\:\s+.+$','',name).strip()
        else:
            name = row.h4.text
            name = name.replace('\n','').strip()
            name = re.sub(r'\s{3}.+','',name).strip()
            name = name.strip()
        names = name.split(',')[0]
        names_.append(names)
        names_ = [x.replace('\n','').strip().title() for x in names_]
        ranks = name.split(',')[1]
        ranks_.append(ranks)
        ranks_ = [x.replace('\n','').strip().title() for x in ranks_]
        area_outliers = row.h4.text
        if area_outliers.find("Fred") > 0:
            area = area_outliers.split(':')[1]
            area = area.replace('\n','').strip()
        else:
            area = row.p.text
            area = area.split(':')[1]
            area = area.replace('\n','').strip()
        area_.append(area)
        
        #Profile
        profile_outliers = row.h4.text
        if profile_outliers.find("Angel Chang") > 0:
            profile = row.p.next_sibling.next_sibling
            profile = profile.a.get('href')
        elif len(row.find_all("a")) == 3:
            profile = row.p.find_next_sibling()
            if profile:
                profile = profile.a.get('href')
            else:
                profile = row.a.get('href')
#         elif len(row.find_all("a")) == 2:
#             profile = row.a.find_next_sibling()
#             if profile:
#                 profile = profile.get('href')
        else:
            profile = row.a.get('href')
        profile_.append(profile)
        
        #Homepage
        if len(row.find_all("a")) == 3:
            homepage = row.p.find_next_sibling()
            if homepage:
                homepage = row.find_all("a")[2].get('href')
        elif len(row.find_all("a")) == 2:
            homepage = row.a.find_next_sibling()
            if homepage:
                homepage = homepage.get('href')
        elif row.a.find_next_sibling():
            pass
        else:
            homepage = None
        homepage_.append(homepage)
    
    for p in enumerate(profile_):
        if p:
            if p[1].startswith('/computing'):
                profile_new = p[1].replace('/computing/','http://www.sfu.ca/computing/')
                i=p[0]
                profile_[i]= profile_new
                
    for h in enumerate(homepage_):
        if h[1]:
            if h[1].startswith('/computing'):
                homepage_new = h[1].replace('/computing/','http://www.sfu.ca/computing/')
                i=h[0]
                homepage_[i]= homepage_new

faculty_df = pd.DataFrame({'name': names_,
'rank': ranks_,
'area': area_,
'profile': profile_,
'homepage': homepage_
})

faculty_df.head(65)
faculty_df.to_csv('faculty_table.csv',index=False)

### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What intersting findings can you get from these visualizations? 

In [246]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

GridBox(id='2585', ...)

Below are some examples:

**Finding 1:** Professor# (29) is more than 3x larger than Associate Professor# (9). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** Homepage has 20.3% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are intersted in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2020+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2020+23-1990 = 53. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 64 profile pages and save each page as a text file. 

In [247]:
# Write your code

import requests
import pandas as pd
df = pd.read_csv('faculty_table.csv')
df_length = len(df)

for i in range(0, df_length):
    page = requests.get(df.iloc[i]['profile'])
    with open(f'faculty_{i}.html',mode='wb') as writer:
        writer.write(page.content)

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [4]:
# write your code here
from bs4 import BeautifulSoup
import re
import pandas as pd

pd.set_option('display.max_rows',1000)

df = pd.read_csv('faculty_table.csv')

df_length = len(df)
name_ = []
gradyear_ = []

for i in range(0, df_length):        
        
    with open(f'faculty_{i}.html',mode='rb') as html_doc:
        soup = BeautifulSoup(html_doc,'html.parser')
        name = soup.find('div',class_='title section')
        name = name.h1.text
        name_.append(name)
        name_ = [x.title() for x in name_]
        data = soup.find_all('div',class_='text parbase section')
        for i, row in enumerate(data):
            text = row.text.strip()
            if text.startswith("Education"):
                gradyear = data[i].text
                if re.findall(r'\d+',gradyear):
                    gradyear = min(re.findall(r'\d+',gradyear))
                else:
                    gradyear = None
                gradyear_.append(gradyear)
                    
                
                
faculty_grad_year = pd.DataFrame({'name':name_,
                                  'gradyear':gradyear_})

faculty_grad_year.head(65)
faculty_grad_year.to_csv('faculty_grad_year.csv',index=False)

,name,gradyear
0,Yagiz Aksoy,2011
1,Bradley Bart,1996
2,Angel Chang,2015
3,Sheelagh Carpendale,1992
4,Leonid Chindelevitch,2006
5,"Cukierman, Diana",1989
6,James Delgrande,1977
7,Toby Donaldson,1992
8,John Edgar,1984
9,Martin Ester,1984


### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [6]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2020+23-df["gradyear"]

plot(df, "age")

Tabs(id='1617', ...)

## Submission

Complete the code in this [notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1.ipynb), and submit it to the CourSys activity `Assignment 1`.

<hr/>
## <span style="color:red">Bonus: Contribute to dataprep</span>

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues) or [link2](https://github.com/sfu-db/DataConnectorConfigs/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature or add data connector for a new website) at [link1](https://github.com/sfu-db/dataprep/pulls) or [link2](https://github.com/sfu-db/DataConnectorConfigs/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 15, 2020`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star.png" width="1000">